### m8.1stan

m8.1stan is the first model in the Statistical Rethinking book (pp. 249) using Stan.

Here we will use Turing's NUTS support, which is currently (2018) the originalNUTS by [Hoffman & Gelman]( http://www.stat.columbia.edu/~gelman/research/published/nuts.pdf) and not the one that's in Stan 2.18.2, i.e., Appendix A.5 in: https://arxiv.org/abs/1701.02434

The TuringModels pkg imports modules such as CSV and DataFrames

In [1]:
using TuringModels

Turing.setadbackend(:reverse_diff);
Turing.turnprogress(false);

┌ Info: [Turing]: global PROGRESS is set as false
└ @ Turing /Users/rob/.julia/packages/Turing/UXxKz/src/Turing.jl:24


Read in the `rugged` data as a DataFrame

In [2]:
d = CSV.read(rel_path("..", "data", "rugged.csv"), delim=';');

Show size of the DataFrame (should be 234x51)

In [3]:
size(d)

(234, 51)

Apply log() to each element in rgdppc_2000 column and add it as a new column

In [4]:
d = hcat(d, map(log, d[Symbol("rgdppc_2000")]));

Rename our col x1 => log_gdp

In [5]:
rename!(d, :x1 => :log_gdp);

Now we need to drop every row where rgdppc_2000 == missing

When this (https://github.com/JuliaData/DataFrames.jl/pull/1546) hits DataFrame it'll be conceptually easier: i.e., completecases!(d, :rgdppc_2000)

In [6]:
notisnan(e) = !ismissing(e)
dd = d[map(notisnan, d[:rgdppc_2000]), :];

Updated DataFrame dd size (should equal 170 x 52)

In [7]:
size(dd)

(170, 52)

Define the Turing model

In [8]:
@model m8_1stan(y, x₁, x₂) = begin
    σ ~ Truncated(Cauchy(0, 2), 0, Inf)
    βR ~ Normal(0, 10)
    βA ~ Normal(0, 10)
    βAR ~ Normal(0, 10)
    α ~ Normal(0, 100)

    for i ∈ 1:length(y)
        y[i] ~ Normal(α + βR * x₁[i] + βA * x₂[i] + βAR * x₁[i] * x₂[i], σ)
    end
end;

Test to see that the model is sane. Use 2000 for now, as in the book.
Need to set the same stepsize and adapt_delta as in Stan...

Use Turing mcmc

In [9]:
posterior = sample(m8_1stan(dd[:log_gdp], dd[:rugged], dd[:cont_africa]),
Turing.NUTS(2000, 1000, 0.95));

┌ Info: [Turing] looking for good initial eps...
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/support/hmc_core.jl:240
[NUTS{Turing.Core.FluxTrackerAD,Union{}}] found initial ϵ: 0.1
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/support/hmc_core.jl:235
┌ Info:  Adapted ϵ = 0.030678798031748687, std = [1.0, 1.0, 1.0, 1.0, 1.0]; 1000 iterations is used for adaption.
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/adapt/adapt.jl:91
[NUTS] Finished with
  Running time        = 235.97243155899974;
  #lf / sample        = 0.0;
  #evals / sample     = 43.528;
  pre-cond. metric    = [1.0, 1.0, 1.0, 1.0, 1.0].


Describe the posterior samples

In [10]:
describe(posterior)

Iterations = 1:2000
Thinning interval = 1
Chains = 1
Samples per chain = 2000

Empirical Posterior Estimates:
              Mean           SD        Naive SE        MCSE         ESS   
       α    9.178085003  0.433940492 0.0097032044 0.02937793424  218.18146
  lf_num    0.000000000  0.000000000 0.0000000000 0.00000000000        NaN
      βA   -1.913381781  0.265290115 0.0059320673 0.01333090874  396.02502
      βR   -0.185937134  0.152922167 0.0034194436 0.01027835143  221.35736
       σ    0.966711775  0.233126529 0.0052128677 0.01423716546  268.12439
 elapsed    0.117986216  0.071785197 0.0016051658 0.00240439519  891.37015
 epsilon    0.030887567  0.026899355 0.0006014879 0.00046378757 2000.00000
eval_num   43.528000000 25.520322479 0.5706517587 0.82232840155  963.12296
     βAR    0.381432189  0.139689366 0.0031235492 0.00388195707 1294.86721
      lp -249.722794495 16.360296709 0.3658273557 1.15128513098  201.93731
  lf_eps    0.030887567  0.026899355 0.0006014879 0.00046378757 2

Fix the inclusion of adaptation samples

In [11]:
posterior2 = MCMCChain.Chains(posterior.value[1001:2000,:,:], names=posterior.names)

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1
Samples per chain = 1000

Union{Missing, Float64}[9.3931 0.0 … -251.957 0.0306788; 9.32455 0.0 … -250.295 0.0306788; … ; 9.12926 0.0 … -247.81 0.0306788; 9.02808 0.0 … -250.511 0.0306788]

Example of a Turing run simulation output

Here's the ulam() output from rethinking

In [12]:
m8_1s_cmdstan = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD       Naive SE       MCSE      ESS
    a  9.22360053 0.139119116 0.0021996664 0.0034632816 1000
   bR -0.20196346 0.076106388 0.0012033477 0.0018370185 1000
   bA -1.94430980 0.227080488 0.0035904578 0.0057840746 1000
  bAR  0.39071684 0.131889143 0.0020853505 0.0032749642 1000
sigma  0.95036370 0.052161768 0.0008247500 0.0009204073 1000

Quantiles:
          2.5%       25.0%       50.0%      75.0%        97.5%
    a  8.95307475  9.12719750  9.2237750  9.31974000  9.490234250
   bR -0.35217930 -0.25334425 -0.2012855 -0.15124725 -0.054216855
   bA -2.39010825 -2.09894500 -1.9432550 -1.78643000 -1.513974250
  bAR  0.13496995  0.30095575  0.3916590  0.47887625  0.650244475
sigma  0.85376115  0.91363250  0.9484920  0.98405750  1.058573750
";

Describe the posterior samples

In [13]:
describe(posterior2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1
Samples per chain = 1000

Empirical Posterior Estimates:
              Mean                   SD                       Naive SE              MCSE         ESS   
       α    9.203900168  0.133746806612780044787314 0.0042294453867045478032138 0.0078263528  292.04381
  lf_num    0.000000000  0.000000000000000000000000 0.0000000000000000000000000 0.0000000000        NaN
      βA   -1.915600284  0.216404341726608773210927 0.0068433061540549879053774 0.0098163546  485.99462
      βR   -0.195288308  0.076545500277554412438263 0.0024205812551412277978946 0.0045698166  280.57037
       σ    0.951301305  0.053488504079339110885982 0.0016914550152591929094159 0.0014307318 1000.00000
 elapsed    0.112241152  0.065458912870544616602331 0.0020699925782943161788052 0.0023053208  806.25949
 epsilon    0.030678798  0.000000000000000024298281 0.0000000000000000007683791 0.0000000000 1000.00000
eval_num   41.344000000 23.148423564327519130756627 0.7320

end of 08/m8.1t.jl#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*